In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# How to Evaluate Propositional Formulas

In this section, we want to show how to evaluate formulas from <em>propositional logic</em>.  First of all we need to discuss how to represent logical formulas.  It is easiest if logical formula are represented as *nested arrays*.  Furthermore, we will represent the propositional operators as unicode symbols according to the following table:
<table border="1px">
    <tr>
        <th>Name</th>
        <th>Symbol</th>
        <th>Unicode</th>
        <th>Unicode Name</th>
    </tr>
    <tr>
        <td><tt>and</tt></td>
        <td>∧</td>
        <td>U+2227</td>
        <td><tt>logical and</tt></td>
    </tr>
    <tr>
        <td><tt>or</tt></td>
        <td>∨</td>
        <td>U+2228</td>
        <td><tt>logical or</tt></td>
    </tr>
    <tr>
        <td><tt>not</tt></td>
        <td>¬</td>
        <td>U+00ac</td>
        <td><tt>not sign</tt></td>
    </tr>
    <tr>
        <td><tt>implies</tt></td>
        <td>→</td>
        <td>U+2192</td>
        <td><tt>rightwards arrow</tt></td>
    </tr>
    <tr>
        <td><tt>if and only if</tt></td>
        <td>↔</td>
        <td>U+2194</td>
        <td><tt>left right arrow</tt></td>
    </tr>
    <tr>
        <td><tt>falsum</tt></td>
        <td>⊥</td>
        <td>U+22a5</td>
        <td><tt>down tack</tt></td>
    </tr>
    <tr>
        <td><tt>verum</tt></td>
        <td>⊤</td>
        <td>U+22a4</td>
        <td><tt>up tack</tt></td>
    </tr>
</table>
For example, the formula 
$$ (p \rightarrow q) \wedge (\neg p \rightarrow q) \rightarrow q $$
will be represented as the following <em>nested array</em>:

```
      ('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q')
```
To enter a unicode character in a string, use the following syntax:
```
     '\U unicode'
```

In [ ]:
const andSymbol = '\u2227';
andSymbol

We can also use the Unicode number to display a unicode character.  As long as the unicode character is located in the *Basic Multilingual Plane*, 
these are the unicode characters `U+0000` to `U+FFFF`, we can write a Unicode character of the form `U+abcd` as `\uabcd`.

In [ ]:
const pq = 'p\u2227q';
pq

For any Unicode code point (including those outside the BMP, up to `U{+10FFFF}`), we can use the `\UXXXXXXXX` escape sequence, where 
`XXXXXXXX` represents the eight hexadecimal digits of the code point (padding with leading zeros if necessary).

In [ ]:
const smiley = '\u{1F600}';
smiley

It is also possible to just copy and paste these symbols.  That is what I usually do.
Another possibility is to use $\LaTeX$ to create these symbols.  We have
* `$\top$` is rendered as $\top$,
* `$\bot$` is rendered as $\bot$,
* `$\neg$` is rendered as $\neg$,
* `$\wedge$` is rendered as $\wedge$,
* `$\vee$` is rendered as $\vee$,
* `$\rightarrow$` is rendered as $\rightarrow$,
* `$\leftrightarrow$` is rendered as $\leftrightarrow$.

As it is quite tedious to type formulas as nested arrays, I have written a parser for propositional formulas, which we will import.
The parser will correctly parse an well-formed string that represents a formula from propositional logic.  However, it will not detect 
certain syntax errors:
  * The parser assumes that the operator $\wedge$ binds stronger than the operator $\vee$.
  
    Therefore, it parses the string $p \vee q \wedge r$ as if it was the formula $p \vee (q \wedge r)$.
    
    According to the conventions of our lecture, this string should be rejected as syntactically invalid.
    
  * The parser assumes that the operator $\leftrightarrow$ is right *associative*.
  
    Therefore, it parses the string $p \leftrightarrow q \leftrightarrow r$ as if it was the formula $p \leftrightarrow (q \leftrightarrow r)$.
    
    According to the conventions of our lecture, this string should be rejected as syntactically invalid.

In [ ]:
import { LogicParser } from './PropositionalLogicParser'

In [ ]:
type Variable = string;
type Formula  = Variable | ['⊤' | '⊥'] | ['¬', Formula] | ['↔' | '→' | '∧' | '∨', Formula, Formula];

Let us test this parser.

In [ ]:
function parse(s: string): Formula {
    const parser = new LogicParser(s);
    return parser.parse();
}

Let us try to parse a complex formula.

In [ ]:
const s = '(p → q) ∧ (¬p → q) → q';
const f = parse(s);
console.log(f);

To see the rest of the formula, we need to ajust the `console.log(f)` slightly.

In [ ]:
console.log(JSON.stringify(f));

Next, we need to decide how to represent a *propositional valuation*.  Mathematically, a *propositional valuation* $\mathcal{I}$ is a function of the form
$$ \mathcal{I}:\mathcal{P} \rightarrow \mathbb{B}  $$
that maps propositional variables to the set of Boolean values 
$\mathbb{B} = \{\texttt{true}, \texttt{false}\}$.  We can represent a propositional  valuation $\mathcal{I}$ as the set of those variables that are evaluated as <tt>true</tt> by $\mathcal{I}$:  
$$ \texttt{repr}(\mathcal{I}) := \bigl\{ v \in \mathcal{P} \mid \mathcal{I}(v) = \texttt{true} \bigr\}$$
For example, the following set would be a propositional valuation for the formula $\texttt{f}$:

In [ ]:
const I1 = new Set(['p', 'q']);

There are 4 possible propositional valuation for the formula $\texttt{f}$.  The other three propositional valuations are as follows:

In [ ]:
const I2 = new Set(['p']);
const I3 = new Set(['q']);
const I4 = new Set<string>();

The function `evaluate` defined below takes two arguments:
   - `F` is a *nested array* representing a *propositional formula*.
   - `I` is a set of *propositional variables* representing a *propositional valuation*.
   
The function returns the value of the propositional formula `F` given the propositional valuation `I`.
When evaluation a formula of the form $f \rightarrow g$ this function uses the equivalence
$$ 
(f \rightarrow g) \;\Leftrightarrow\; \neg f \vee g. 
$$ 

In [ ]:
console.log(false < true);

In [ ]:
function evaluate(F: Formula, I: Set<string>): boolean {
    if (typeof F === 'string')
        { return I.has(F);}
    if (F[0] === '⊤')   return true;
    if (F[0] === '⊥')   return false;
    if (F[0] === '¬') { return !evaluate(F[1], I);}
    if (F[0] === '∧') { return  evaluate(F[1], I) &&   evaluate(F[2], I);}
    if (F[0] === '∨') { return  evaluate(F[1], I) ||   evaluate(F[2], I);}
    if (F[0] === '→') { return !evaluate(F[1], I) ||   evaluate(F[2], I);}
    if (F[0] === '↔') { return  evaluate(F[1], I) ===  evaluate(F[2], I);}
    return false;
}

Let's test this function for the formula 
$$ f := (p \rightarrow q) \wedge (\neg p \rightarrow q) \rightarrow q. $$

In [ ]:
JSON.stringify(f);

In [ ]:
for (const I of [I1, I2, I3, I4]) {
    console.log(`evaluate(${JSON.stringify(f)}, {${Array.from(I).join(', ')}} = ${evaluate(f, I)}`);
}

As the formula <tt>f</tt> is true for **every** possible propositional valuation `I`, it is a *tautology*.